In [48]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform

In [49]:
def get_reform(amount):
    return Reform.from_dict({
    "gov.contrib.crfb.ss_credit.in_effect": {
        "2026-01-01.2100-12-31": True
    },
    "gov.contrib.crfb.ss_credit.amount.SINGLE": {
        "2026-01-01.2100-12-31": amount
    },  
    "gov.contrib.crfb.ss_credit.amount.JOINT": {
        "2026-01-01.2100-12-31": amount
    },  
    "gov.contrib.crfb.ss_credit.amount.SEPARATE": {
        "2026-01-01.2100-12-31": amount
    },  
    "gov.contrib.crfb.ss_credit.amount.HEAD_OF_HOUSEHOLD": {
        "2026-01-01.2100-12-31": amount
    },  
    "gov.contrib.crfb.ss_credit.amount.SURVIVING_SPOUSE": {
        "2026-01-01.2100-12-31": amount
    },  
    "gov.irs.deductions.senior_deduction.amount": {
        "2026-01-01.2100-12-31": 0
    },
    "gov.irs.social_security.taxability.rate.base": {
        "2026-01-01.2100-12-31": 0.85
    },
    "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
        "2026-01-01.2100-12-31": 0
    },
    "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
        "2026-01-01.2100-12-31": 0
    },
    "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
        "2026-01-01.2100-12-31": 0
    },
    "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
        "2026-01-01.2100-12-31": 0
    },
    "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
        "2026-01-01.2100-12-31": 0
    },
    }, country_id="us")


In [61]:
AMOUNTS = [300, 600, 900, 1_200, 1_500]

In [62]:
baseline = Microsimulation()
baseline_income = baseline.calculate("household_net_income", period=2026)


In [63]:
results = []

In [64]:
for amount in AMOUNTS:
    reform = get_reform(amount)
    reformed = Microsimulation(reform=reform)
    reformed_income = reformed.calculate("household_net_income", period=2026)
    difference_income = reformed_income - baseline_income
    dif = difference_income.sum() / 1e9
    results.append([amount, f'${round(dif, 2)}'])

In [65]:
import pandas as pd

In [66]:
df = pd.DataFrame(results, columns=['Credit Amount', 'Cost (Billions)'])
print(df)


   Credit Amount Cost (Billions)
0            300         $-31.11
1            600         $-21.33
2            900         $-12.54
3           1200          $-4.64
4           1500           $2.41


In [67]:
from policyengine_us import Simulation

In [68]:
def household_situation():
    
  return {
    "people": {
      "you": {
        "age": {
          "2026": 65
        },
        "employment_income": {
          "2026": 60_000
        },
        "social_security_retirement": {
          "2026": 15000
        }
      },
      "your partner": {
        "age": {
          "2026": 67
        },
        "employment_income": {
          "2026": 60_000
        },
        "social_security_retirement": {
          "2026": 5000
        }
      }
    },
    "tax_units": {
      "your tax unit": {
        "members": [
          "you",
          "your partner"
        ]
      }
    },
    "households": {
      "your household": {
        "members": [
          "you",
          "your partner"
        ],
        "state_name": {
          "2026": "TX"
        }
      }
    }
  }



In [69]:
baseline_sim = Simulation(
    situation=household_situation(),
)
baseline_income = baseline_sim.calculate("household_net_income", 2026)

In [70]:
household_results = []
for amount in AMOUNTS:
    reform = get_reform(amount)
    reformed_sim = Simulation(
        situation=household_situation(),
        reform=reform,
    )
    reformed_income = reformed_sim.calculate("household_net_income", 2026)
    difference = reformed_income - baseline_income
    household_results.append([amount, f'${difference[0]}'])


In [72]:
household_df = pd.DataFrame(household_results, columns=['Credit Amount', 'Net Income Change'])
print(household_df)


   Credit Amount Net Income Change
0            300          $-1255.0
1            600           $-655.0
2            900            $-55.0
3           1200            $280.0
4           1500            $580.0
